In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random 
import numpy as np 
from PIL import Image
import os


size_x = 1280
size_y = 720
target_path =  'hd.jpg'
components = [
    'kersti/0.jpg',
    'kersti/1.jpg',
    'kersti/2.jpg'
]
def open_image():
    print("open")
#     imgplot = plt.imshow(img)

    
def draw_one_image_random(result):
    x,y,w,h = random.randrange(size_x),random.randrange(size_y), random.randrange(10,300), random.randrange(10,300)
    img = Image.open(random.choice(components))
    if (x+w < size_x) and (y+h < size_y):
        resized = img.resize((w,h), Image.ANTIALIAS)
        result.paste(resized, (x, y, x+w, y+h))

def mutate( init_pic = None):
    if init_pic == None:
        result = Image.new("RGB", (size_x, size_y))
    else:
        result = init_pic.copy()
    for i in range(100):
        draw_one_image_random(result)
    return result

def save_to_file(result, path):
    result.save(os.path.expanduser(path))
    pil_im = Image.open(path)
    display(pil_im)

# result = mutate()
# display(result)

# result = mutate(result)
# display(result)

# result = mutate(result)
# display(result)

# save_to_file(result, "result.jpg")


In [ ]:
import imagehash

def fitness(imga,imgb):
#     hash0 = imagehash.average_hash(Image.open('hd.jpg')) 
#     hash1 = imagehash.average_hash(Image.open('result.jpg')) 
    hash0 = imagehash.average_hash(imga) 
    hash1 = imagehash.average_hash(imgb) 
    return hash0-hash1

In [ ]:
target_img = Image.open(target_path)
def init_first():
    #get first 100 child
    childs = []  
    for i in range(100):
        tmp = mutate()
        print(".",end="")
        childs.append(tmp)
    return childs

def get_fitness_score(generation):
    fitness_scores = []
    for i in generation:
        fitness_score = fitness(i, target_img)
        fitness_scores.append(fitness_score)
    return fitness_scores
        
def run():
    generation = init_first()

    
def run_iterate(generation):
    fitness_scores = get_fitness_score(generation)
    print(fitness_scores)
    sorted_index = np.argsort(fitness_scores)
    #choose best 20 from the generation
    bestX = sorted_index[:21]
    print(bestX)
    #save best one
    
    #every child generate 50 child
    new_generation = []
    
    #insert best 10 of itself
    for i in range(20):
        fitness_score = fitness(generation[bestX[i]], target_img)
        new_generation.append(generation[bestX[i]])
    
    for i in range(20):
        #mutate 5 time
        for j in range(5):
            print(".",end="")
            new_generation.append(mutate(generation[bestX[i]]))
    
    return new_generation
    #repeat
    
# run()

In [ ]:
generation = init_first()
fitness_scores = get_fitness_score(generation)
sorted_index = np.argsort(fitness_scores)
print(fitness_scores)
print("best fitness : ",fitness_scores[sorted_index[0]])
display(generation[sorted_index[0]])

In [ ]:
# display(target_img)
# display(generation[sorted_index[0]])
for i in range(10):
    print("iterate : ",i)
    generation = run_iterate(generation)
    fitness_scores = get_fitness_score(generation)
    print(fitness_scores)
    sorted_index = np.argsort(fitness_scores)
    print("best fitness : ",fitness_scores[sorted_index[0]])
    display(generation[sorted_index[0]])

In [ ]:
for i in range(20):
    print("iterate : ",10+i)
    generation = run_iterate(generation)
    fitness_scores = get_fitness_score(generation)
    print(fitness_scores)
    sorted_index = np.argsort(fitness_scores)
    print("best fitness : ",fitness_scores[sorted_index[0]])
    display(generation[sorted_index[0]])

In [ ]:
for i in range(150):
    print("iterate : ",30+i)
    generation = run_iterate(generation)
    fitness_scores = get_fitness_score(generation)
    print(fitness_scores)
    sorted_index = np.argsort(fitness_scores)
    print("best fitness : ",fitness_scores[sorted_index[0]])
    display(generation[sorted_index[0]])

In [ ]:
save_to_file(generation[sorted_index[0]], "result-150-gen.jpg")